In [32]:
import pandas as pd
import re
import datetime as dt
import os
import numpy as np

In [101]:
def get_timestamp(filename):
    rpyear = re.compile(r"(\.h2\.)(.*?)(-)")
    rpdaymonth = re.compile(r"(-)(.*?)(\d{5}\.)")
    year=int(rpyear.search(filename).groups()[1])
    tmp=rpdaymonth.search(filename).groups()[1].split('-')
    month=int(tmp[0])
    day=int(tmp[1])
    return dt.date(year,month,day)    
    
def get_boxes_for_nc_file(filepath, path_to_csv_file="/global/cscratch1/sd/racah/climate_data/climo/csv_labels/new_labels.csv"):
    ims_per_file = 8
    boxes = [ims_per_file*[]]
    filename = os.path.basename(filepath)
    
    coord_keys = ["xmin", "xmax", "ymin", "ymax"]
    cls_key = "category"
    
    labeldf = pd.read_csv(path_to_csv_file)
    
    
    ts = get_timestamp(filename)

    filedf = labeldf.ix[ (labeldf.month==ts.month) & (labeldf.day==ts.day) & (labeldf.year==ts.year) ].copy()


    time_steps = [ int(num) for num in list(filedf["time_step"])]

    box_coords = filedf[["xmin", "xmax", "ymin", "ymax", "category"]].values

    ims_per_file = 8
    boxes = [[] for i in range(ims_per_file)]
    time_step_boxes_pairs = zip(time_steps, box_coords)
    for time_step, box in time_step_boxes_pairs:
        boxes[time_step].append(list(box))
    
    return boxes
    
    

In [85]:
if __name__ == "__main__":
    filepath = "/global/cscratch1/sd/racah/climate_data/climo/images/cam5_1_amip_run2.cam2.h2.1985-09-23-10800.nc"

    boxes = get_boxes_for_nc_file(filepath)